# Projeto 1 - Ciência dos Dados

Nome: Cesar Ades

Nome: Jonathan Sutton

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from emoji import UNICODE_EMOJI

ModuleNotFoundError: No module named 'emoji'

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [ ]:
import os

filename = 'ps4.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com o trabalho!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [ ]:
train = pd.read_excel(filename)
train.head(5)

In [ ]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.


Nosso produto é o Play Station 4 que é um console para videogames fabricado pela Sony, tem diversos jogos e utiliza controles bluetooth.Consideramos tweets relevantes aqueles que envolviam informações sobre o preço, local de fabricação, performance do produto, jogos que podem ser usados, lançamentos de jogos e falhas que ocorrem com o console e e os produtos relacionados a ele.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
#Definindo as variáveis como categóricas
train['Relevância'] = train['Relevância'].astype('category')
test['Relevância'] = test['Relevância'].astype('category')
train['Relevância'].cat.categories = ['Não relevante','Relevante']  
test['Relevância'].cat.categories = ['Não relevante','Relevante']

In [ ]:
test.head(5)

In [ ]:
#Função que separa os emojis
def separate_emoji(text): #separa emojis dos outros caracteres
    return ''.join(' ' + char if char in UNICODE_EMOJI else char for char in text).strip()

In [ ]:
#Função que limpa os tweets
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;/@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', separate_emoji(text))
    return text_subbed    

In [ ]:
def treinamento(data):
    p_relevantes = dict()
    p_irrelevantes = dict()
    p_possiveis = set()
    for index,row in data.iterrows():
        tweet = row.Treinamento
        for palavra in cleanup(tweet).split():
            p_possiveis.add(palavra)
            if row.Relevância == 'Relevante':
                p_relevantes[palavra] = p_relevantes.get(palavra,0)+1
            else:
                p_irrelevantes[palavra] = p_irrelevantes.get(palavra,0)+1
    return (p_possiveis,p_relevantes,p_irrelevantes)

In [ ]:
p_count = treinamento(train)

In [ ]:
#Imprime informações relevantes sobre o excel.
resultados_treino = train.Relevância.value_counts(True)
relevantes_treino = resultados_treino[0]
irrelevantes_treino = resultados_treino[1]
resultados_brutos_treino = train.Relevância.value_counts()
relevantes_brutos_treino = resultados_brutos_treino[0]
irrelevantes_brutos_treino = resultados_brutos_treino[1]
tweets_totais_treino = relevantes_brutos_treino + irrelevantes_brutos_treino
resultados_teste = test.Relevância.value_counts(True)
relevantes_teste = resultados_teste[0]
irrelevantes_teste = resultados_teste[1]
resultados_brutos_teste = test.Relevância.value_counts()
relevantes_brutos_teste = resultados_brutos_teste[0]
irrelevantes_brutos_teste = resultados_brutos_teste[1]
tweets_totais_teste = relevantes_brutos_teste + irrelevantes_brutos_teste

print('Tweets relevantes em Treinamento: {0}'.format(relevantes_brutos_treino))
print('Tweets irrelevantes em Treinamento: {0}'.format(irrelevantes_brutos_treino))
print('Tweets totais em Treinamento: {0}'.format(tweets_totais_treino))
print('Porcentagem de Tweets relevantes em Treinamento: {0:.2f}%'.format(relevantes_treino*100))
print('Porcentagem de Tweets irrelevantes em Treinamento: {0:.2f}%'.format(irrelevantes_treino*100))
print('Tweets relevantes em Teste: {0}'.format(relevantes_brutos_teste))
print('Tweets irrelevantes em Teste: {0}'.format(irrelevantes_brutos_teste))
print('Tweets totais em Teste: {0}'.format(tweets_totais_teste))
print('Porcentagem de Tweets relevantes em Teste: {0:.2f}%'.format(relevantes_teste*100))
print('Porcentagem de Tweets irrelevantes em Teste: {0:.2f}%'.format(irrelevantes_teste*100))

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [2]:
#Função Naive Bayes
def classificador_de_tweets(data,words):
    p_possiveis,p_relevantes,p_irrelevantes = words
    classificacoes = []
    for index,row in data.iterrows():
        tweet = row.Teste
        prob_relevante,prob_irrelevante = 1,1
        for palavra in cleanup(tweet).split():
            prob_relevante *= (p_relevantes.get(palavra,0)+1)/(len(p_possiveis) + sum(p_relevantes.values()))
            prob_irrelevante *= (p_irrelevantes.get(palavra,0)+1)/(len(p_possiveis) + sum(p_irrelevantes.values()))
        classificacoes.append('Não relevante' if prob_relevante < prob_irrelevante else 'Relevante')
    return classificacoes

In [3]:
#Adiciona coluna no Dataframe test
test['Classificação'] = classificador_de_tweets(test,p_count)
test.head(10)

NameError: name 'test' is not defined

Tabelas para ajudar a visualizar resultados.

In [4]:
pd.crosstab(test.Relevância,test.Classificação)

NameError: name 'test' is not defined

In [5]:
(pd.crosstab(test.Relevância,test.Classificação,normalize = 'index')*100).round(decimals=3)

NameError: name 'test' is not defined

In [6]:
(pd.crosstab(test.Relevância,test.Classificação,normalize = True)*100).round(decimals=3)

NameError: name 'test' is not defined

In [7]:
print('Tivemos 65 tweets verdadeiros positivos, com 76,471% dos tweets que foram classificados como relevantes')
print('Tivemos 20 tweets falsos positivos, com 23,529% dos tweets que foram classificados como relevantes')
print('Tivemos 70 tweets verdadeiros negativos, com 60,87% dos tweets que foram classificados como irelevantes')
print('Tivemos 45 tweets falsos negativos, com 39,13% dos tweets que foram classificados como irrelevantes')
print('A porcentagem de erro do classificador foi de 32,5%.')

Tivemos 65 tweets verdadeiros positivos, com 76,471% dos tweets que foram classificados como relevantes
Tivemos 20 tweets falsos positivos, com 23,529% dos tweets que foram classificados como relevantes
Tivemos 70 tweets verdadeiros negativos, com 60,87% dos tweets que foram classificados como irelevantes
Tivemos 45 tweets falsos negativos, com 39,13% dos tweets que foram classificados como irrelevantes
A porcentagem de erro do classificador foi de 32,5%.


___
### Concluindo

Ao ver os resultados do projeto, concluimos que nosso objetivo foi atingido. Isso porque, o classificador acertou a relevância em 67,5% dos tweets, porém o classificador considerou 32,5% o contrário da nossa classificação. Isso pode ter se dado por conta de a nossa classificação não ter sido rigorosa o suficiente ao classificar o que era relevante e o que era irrelevante. Algo que poderia ser feito para aumentar ainda mais a perfomance do classificador seria ser mais criterioso na hora de classificar os tweets como relevantes ou irrelevantes.

As mensagens que contém sarcasmo não são classificadas pelo classificador como tais, pois o significado intendido pelo usuário não é compreendido. Isso acontece por conta do classificador apenas verificar se ele seria releveante ou não, e não analisar os significados reais da palavra. Mensagens com dupla negação tambem não são classificadas como tal pelo classificador, pois ele analisa a palavra individualmente e não a possição que ocupa no texto. Um plano de expansão incluiria um aumento significativo das palavras que seriam recolhida, para que exista a possibilidade de classificar os tweets de maneira mais precisa. Além disso, novas técnicas seriam pesquisadas e implementadas para tentar distinguir os tweets reais e os que tem dupla negação e sarcasmo.

O nosso projeto deveria ser financiado pois, com mais tempo de trabalho, poderíamos aperfeiçoar o nosso classificador para entender mensagens de sarcasmo, dupla negação, etc. Além disso, criaríamos mais categorias (como por exemplo, a opção "neutro") para tornar o classificador mais preciso. Em mais tempo de trabalho, seria possível usar esse classificador em tempo real!

Não seria possível usar alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets, pois os tweets para treinamento servem para ensinar o computador como ler as mensagens relevantes e irrelevantes. 

O classificador de Naive Bayes poderia ser usado para vários outros casos, como por exemplo recomendar um filme ou uma música para um usuário, dependendo do que ele vai selecionando. Assim, o classificador calcularia o filme ou música com a maior probabilidade de ser aprovada, levando em conta questões como gênero, duração, atores envolvidos, estilo da música, cantor/banda, etc.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**